In [1]:
import fmlib
import numpy as np
from math import *
import grading_tools


# Exercises

## Exercise

Write a function `put_hedging_error` that simulates replicating a put option using the Black-Scholes strategy and returns
a single number: the root mean square error. You should use the functions in `fmlib.py` rather than write all the code from scratch.

In [2]:
def put_hedging_error( S0, K, T, r, mu, sigma, n_steps, n_paths):
    ### BEGIN SOLUTION
    S, times = fmlib.simulate_gbm_paths(S0,mu,sigma,T,n_steps,n_paths)
    Pi = np.zeros(S.shape)
    t=times[0]
    V = fmlib.black_scholes_put_price(S0,0,K,T,r,sigma)
    Pi[:,0] = V
    dt = T/n_steps #...
    for i in range(0,len(times)-1):
        t = times[i]
        S_t=S[:,i]
        q_t = fmlib.black_scholes_put_delta(S_t,t,K,T,r,sigma)
        B_t = Pi[:,i]-q_t*S_t
        B_with_interest = B_t*exp(dt*r)
        S_tplusdt=S[:,i+1]
        new_stock_value=q_t*S_tplusdt
        Pi[:,i+1]=B_with_interest+new_stock_value

    S_T = S[:,-1]
    final_balance=Pi[:,-1]
    error = final_balance - np.maximum(K-S_T,0)
    rms_error = sqrt( np.mean(error**2) )
    return rms_error
    ### END SOLUTION

In [3]:
error = put_hedging_error( 100, 80, 1, 0.02, 0.08, 0.2, 365, 500)
grading_tools.check( '1235ef', error, tol=0.05)
grading_tools.auto_marking_message()

## Exercise

Suppose that a trader sells a put option at the Black-Scholes price but chooses not to hedge it at all. Suppose that $K=100$, $S_0=100$, $T=1$, $r=0.02$, $\mu=0.2$ and $\sigma=0.1$. Compute the average profit of the trader, it should be positive.

If this trader makes a profit on average, does this mean that the Black-Scholes price is too high?

In [4]:
def compute_no_hedging_profit( S0, K, T, r, mu, sigma, n_paths):
    ### BEGIN SOLUTION
    S, times = fmlib.simulate_gbm_paths(S0,mu,sigma,T,1,n_paths)
    V = fmlib.black_scholes_put_price(S0,0,K,T,r,sigma)
    final_balance = V*exp(r*T)
    S_T = S[:,-1]
    profit = final_balance - np.maximum(K-S_T,0)
    return profit
    ### END SOLUTION

In [5]:
n_paths = 1000000
profit_vec = compute_no_hedging_profit( 100, 100, 1, 0.02, 0.2, 0.1, 1000000 )
assert( np.mean( profit_vec )> 0)
grading_tools.check( '12djfeg', 0*error, tol=0.05)
grading_tools.auto_marking_message()

Consider the case of a trader just buying stock. It will grow *on average* at a rate faster than $r$ since $\mu>r$, so there is no reason why a *risky* strategy shouldn't allow you to make money on average. In this example, the trader is effectively gambling on the stock price increasing so they will benefit from the fact $\mu>r$ in just the same way. It would be strange if the strategy made a profit with certainty, but it does not.